<a href="https://colab.research.google.com/github/JuliaHdzQ/UFV_VisualizaciondeDatos/blob/main/clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO HISTÓRICO DE VENTAS

> Julia Hernández Quintas

> Visualiazación de datos / 4º Ingeniería Matemática

> ## Introducción  

La visualización de datos es el proceso de representar información utilizando gráficas, y distintas herramientas visuales, que faciliten tanto su comprensión, como la transmisión del mensaje deseado.

En un mundo donde la cantidad de datos aumenta exponencialmente, la visualización de datos se ha convertido en una herramienta clave para su comprensión y desarrollo.

> ## Objetivos y mensaje

Con este proyecto de visualización de datos se pretende analizar y comprender el comportamiento de un conjunto de ventas y la precisión de las predicciones de nuevas ventas, en función del pais, periodo, marca ..., con el fin de poder mejorar las decisiones estratégicas en ventas.

Para ello, realizaremos estudios sobre sus distribuciones, tendencias, estacionalidad, relaciones ... y otros factores que puedan ayudarnos a la obtención de información útil sobre dichas ventas.

Para ello, tendremos como objetivo responder a las siguiente preguntas propuestas:


1. ¿Como se distribuyen las ventas realizadas?
    - Cada pais
    - Cada mes y año
    - Cada marca

2. Cual es la tendencia y estacionalidad de:
    - Todas las ventas del pais con menos ventas
    - La marca con mas ventas

3. ¿Cuales son las predicciones hechas en España y como de buenas son ?

> ## Comprensión de variables

1. **COUNTRY**: pais al que pertenece el dato de la venta o predicción

2. **SUBBRAND**: marca de la venta

3. **YEAR**: año de las ventas reales / el año sobre el que hago la prediccion

4. **MONTH**: mes de las ventas reales / el mes sobre el que hago la prediccion

5. **SCENARIO**: puede tener dos valores

6. **FORECAST**: prediccion realizada para ese mes y ese año (entonces tendremos valores en los siguientes dos campos)

    - *Actual* : venta real en ese mes y ese año
    - *Forecast* : en que mes se hizo la prediccion, despues de la p, viene el numero del mes en que se hizo la prediccion mas uno. Es decir AI_P02F es enero

7. **FORECAST_YEAR**: el año en que se hizo la predicción

8. **AMOUNT**: la cantidad

> ## Carga y Comprensión de la Base de Datos

El siguiente paso será cargar la base de datos, y visualizarla, lo que nos permitirá ver de un primer vistazo la cantidad de atributos, el tipo de datos que compone cada uno de ellos, la cantidad de registros ...

In [18]:
#pip install plotly_express

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [20]:
from google.colab import drive
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas_3.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)
data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05


> **VALORES NULOS O REPETIDOS**

Antes de comenzar a evaluar y trabajar sobre la base de datos, vamos a comprobar si existen valores nulos o registros repetidos.

Gracias a la comprensión de los atributos que conforman nuestra base de datos, sabemos que para los registros de ventas reales, es decir aquellos donde el atributo SCENARIO = actual, no tendran valores en las dos siguientes variables (forecast, forecastyear).

Así pues, realizamos el análisis de valores nulos en nuestra base con la intención de que solo obtengamos estos valores en los puntos mencionados, y de no ser así valorar si eliminamos o no los registros donde aparecen.

In [21]:
nulos = data.isnull().sum()
print(nulos)

COUNTRY            0
SUBBRAND           0
YEAR               0
MONTH              0
SCENARIO           0
FORECAST         900
FORECAST_YEAR    900
AMOUNT             0
dtype: int64


En cuanto a los registros repetidos, vamos a eliminar los registros de prediccines repetidos, pues no tiene sentido tener dos predicciones iguales.
En el caso de ventas reales, no existen registros repetidos.

In [22]:
# Buscamos todos los resgistros diferentes
conteo_duplicados = data.groupby(list(data.columns)).size()

# Filtramos solo los registros que aparecen mas de una vez
duplicados = conteo_duplicados[conteo_duplicados > 1]
print(duplicados)

COUNTRY        SUBBRAND     YEAR  MONTH  SCENARIO     FORECAST  FORECAST_YEAR  AMOUNT       
Great Britain  7up (L3)     2023  7      AI_forecast  AI_PF     2023.0         399050.441302    2
                                  8      AI_forecast  AI_P02F   2023.0         380865.044383    2
                                                      AI_PF     2023.0         368000.684443    2
                                  9      AI_forecast  AI_P02F   2023.0         358295.681564    2
                                                      AI_P03F   2023.0         369530.056459    2
                                                                                               ..
Norway         Lipton (L3)  2024  6      AI_forecast  AI_P12F   2023.0         0.000000         2
                                  10     AI_forecast  AI_P12F   2023.0         0.000000         2
                                  11     AI_forecast  AI_P12F   2023.0         0.000000         2
                         

Guardamos la base de datos sin registros de ventas reales repetidos.

In [23]:
data = data.drop_duplicates()

> **PAISES Y MARCAS ÚNICAS**

Vamos a ver cuales son los valores unicos de paises y de marcas, es decir que paises y marcas son las que aparecen en nuestra base de datos:


In [24]:
paises = data['COUNTRY'].unique()
marcas = data['SUBBRAND'].unique()
N = len(paises)
N2 = len(marcas)
fig = go.Figure(data=[go.Table(header=dict(values=["<b>Información</b>", "<b>Recuento</b>", "<b>Listado</b>"], fill_color='lightblue', align='center', font=dict(size=12, color='black'), height=30), cells=dict(values=[["Países", "Marcas"], [N, N2], [', '.join(paises), ', '.join(marcas)]], fill_color='lightgray', align='center', font=dict(size=11), height=25))])
fig.update_layout(width=800)
fig.update_traces(columnwidth=[0.4, 0.2, 0.8])
fig.show()

> **REGISTROS DE CADA PAIS / MARCA**

Vamos a hacer ahora una rapida evaluación del numero de registros (frecuencia) que tiene cada país, y cada marca. Para ello utilizaremos dos histogramas:


In [25]:
import plotly.express as px
#  Subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'pie'}, {'type': 'pie'}]], subplot_titles=("Paises", "Marcas"))

# Gráficos de pastel para 'COUNTRY' y 'SUBBRAND'
tarta_country = px.pie(data, names='COUNTRY')
tarta_subbrand = px.pie(data, names='SUBBRAND')

fig.add_trace(tarta_country.data[0], row=1, col=1)
fig.add_trace(tarta_subbrand.data[0], row=1, col=2)

# Mostrar la figura con los gráficos de pastel
fig.update_layout(title_text="Comparación de Diagramas de Pastel")
fig.show()

Vamos a ver también con cuantas marcas trabaja cada país, si todos ellos trabajan con todas las marcas, solo con algunas ...

In [26]:
# Contamos las marcas distintas en cada país
grouped_data = data.groupby('COUNTRY')['SUBBRAND'].nunique().reset_index()
grouped_data.columns = ['COUNTRY', 'Unique Brands']

# Creamos el histograma
fig = px.bar(grouped_data, x='COUNTRY', y='Unique Brands', title="Número de Marcas Distintas por País")

# Ajustamos color y ancho de barras
fig.update_traces(marker_color='rgba(135, 206, 235, 0.8)',
                  marker_line_color='rgba(0, 0, 0, 0.3)',
                  marker_line_width=0.5, width=0.6)

fig.show()

Vemos que son varios los paises que si trabajan con todas las marcas, y que el país que trabaja con menos marcas es Italia, que lo hace con solo 3

> **Nº ACTUALS Y FORECAST**

Vamos a buscar cuantos registros son de ventas reales o actuals, y cuantos de los registros son valores de predicciones o forecast:

In [27]:
# Comprobamos que solo hay los dos valores de SCENARIO esperados
v = data['SCENARIO'].unique()

num_ventas = (data['SCENARIO'] == 'actual').sum()  # Suma True (1) para 'A'
print('Numero total de ventas reales:', num_ventas)

num_predicciones = (data['SCENARIO'] == 'AI_forecast').sum()
print('Numero total de predicciones:', num_predicciones)

Numero total de ventas reales: 900
Numero total de predicciones: 17103


In [28]:
fig = go.Figure(data=[
    go.Bar(name='Registros', y=['Ventas', 'Predicciones'], x=[num_ventas, num_predicciones], orientation='h',marker=dict(color='rgba(255, 165, 0, 0.8)'))])

fig.update_layout(
    title="Comparación Registros de Ventas y Predicciones",
    xaxis_title="Nº de Registros", yaxis_title="Scenario",)

fig.show()

En nuestros datos, tenemos una cantidad muchísmo mas elevada de registros de **predicciones** que de *ventas reales* como se demuestra perfectamente empleando este gráfico de barras horizontal.

> **HORIZONTE DE PREVISIÓN**

En este punto, queremos estudiar el horizonte de previsión, que nos permitirá saber hasta qué fecha se realizan las predicciones, y cuántos períodos cubre (en este caso meses)

Sabiendo que el horizonte de previsión es igual para todas las marcas y países, lo estudiaremos para el caso particular de:: Spain, Lipton (L3)


In [29]:
# Definir el país y la bebida específicos, y el mes
pais_especifico = "Spain"
bebida_especifica = "Lipton (L3)"
mes_especifico = "AI_P02F"

horizonte_de_precision = ((data['COUNTRY'] == pais_especifico) &
                   (data['SUBBRAND'] == bebida_especifica) &
                   (data['FORECAST'] == mes_especifico)).sum()

print('Horizonte de precision:', horizonte_de_precision)

Horizonte de precision: 18


EXPLICACION EXTRA MENDIETA


> **HISTÓRICO DE DATOS DE ACTUALS Y FORECAST**


Quermos evaluar el histórico de ventas reales, y también de predicciones.


**VENTAS REALES**: Empezaremos por las ventas reales, y lo haremos primero para el año 2023, y a continuación para el año 2024. En este gráfico podremos ver la evolución temporal de las ventas reales (en que mes comienzan y cuando finalizan) y como evoluciona el número de ventas por mes. Para ello hemos empleado el siguiente gráfico de lineas:

In [30]:
# Especificamos datos de VENTAS reales
scenario_especifico = "actual"

anio_2023 = 2023
ventas_2023 = data[(data['SCENARIO'] == scenario_especifico) & (data['YEAR'] == anio_2023)]
ventas_por_mes_2023 = ventas_2023.groupby('MONTH').size().reindex(range(1, 13), fill_value=0).reset_index(name='NUM_VENTAS')

anio_2024 = 2024
ventas_2024 = data[(data['SCENARIO'] == scenario_especifico) & (data['YEAR'] == anio_2024)]
ventas_por_mes_2024 = ventas_2024.groupby('MONTH').size().reindex(range(1, 13), fill_value=0).reset_index(name='NUM_VENTAS')

fig = make_subplots(rows=1, cols=2, subplot_titles=("Ventas por Mes en 2023", "Ventas por Mes en 2024"))

# Gráfico de líneas para 2023
linea_2023 = go.Scatter(x=ventas_por_mes_2023['MONTH'], y=ventas_por_mes_2023['NUM_VENTAS'], mode='lines', name='2023')
fig.add_trace(linea_2023, row=1, col=1)

# Gráfico de líneas para 2024
linea_2024 = go.Scatter(x=ventas_por_mes_2024['MONTH'], y=ventas_por_mes_2024['NUM_VENTAS'], mode='lines', name='2024')
fig.add_trace(linea_2024, row=1, col=2)

fig.update_layout(
    title="Comparación de Ventas por Mes en 2023 y 2024",
    xaxis_title="Mes",
    yaxis_title="Nº de Ventas",)

fig.show()

A partir del gráfico podemos sacar varias conclusiones.

La primera de ellas es que los datos de ventas reales van desde enero de 2023, hasta Agosto de 2024. NO hay registros de ventas reales del último cuatrimestre de 2024.

También vemos que en ambos casos, sin tener en cuenta los meses donde no hay ventas, el mínimo número de ventas es 44, y el máximo es 45 en 2023, y 46 en 2024.

Haremos lo mismo ahora para las predicciones o forecast:

**PREDICCIONES:** en este punto vamos a analizar la periocidad de las predicciones. Veremos aquí el nº de predicciones hechas en cada mes para cada uno de los años a estudiar, y como evolucionan a lo largo de los meses.

In [31]:
scenario_especifico = "AI_forecast"

anio_2023 = 2023
forecast_2023 = data[(data['SCENARIO'] == scenario_especifico) & (data['YEAR'] == anio_2023)]
forecast_por_mes_2023 = forecast_2023.groupby('MONTH').size().reindex(range(1, 13), fill_value=0).reset_index(name='NUM_PREDICCIONES')

anio_2024 = 2024
forecast_2024 = data[(data['SCENARIO'] == scenario_especifico) & (data['YEAR'] == anio_2024)]
forecast_por_mes_2024 = forecast_2024.groupby('MONTH').size().reindex(range(1, 13), fill_value=0).reset_index(name='NUM_PREDICCIONES')

fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Predicciones por Mes en 2023", "Predicciones por Mes en 2024"))

# Gráfico de líneas para 2023
linea_2023 = go.Scatter(x=forecast_por_mes_2023['MONTH'], y=forecast_por_mes_2023['NUM_PREDICCIONES'], mode='lines', name='2023')
fig2.add_trace(linea_2023, row=1, col=1)

# Gráfico de líneas para 2024
linea_2024 = go.Scatter(x=forecast_por_mes_2024['MONTH'], y=forecast_por_mes_2024['NUM_PREDICCIONES'], mode='lines', name='2024')
fig2.add_trace(linea_2024, row=1, col=2)

fig2.update_layout(
    title="Comparación de Predicciones por Mes en 2023 y 2024",
    xaxis_title="Mes",
    yaxis_title="Nº de Predicciones",)

fig2.show()

La gráfica elegida nos ayuda a ver la tendencia en el nº de predicciones realizadas en función del tiempo. Vemos como en el año 2023, esta evolución es un crecimiento lineal.

Luego en 2024, el crecimiento se estabiliza y se mantiene constante y cercano a 950 durante la primera mitad de año.

El segundo semestre del año 2024, la tendencia pasa a ser negativa y empieza a decrecer linealmente el nº de predicciones hasta finalizar el año.

> **FORECASTS DISTINTOS**



> ## Distribución de las ventas realizadas


> **DISTRIBUCIÓN DE LAS VENTAS POR PAÍS**

Vamos a analizar la distribución de las ventas en cada país.

In [32]:
data_actual = data[(data['SCENARIO'] == 'actual')]

ventas_por_pais = data_actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index(name='VENTAS')
ventas_por_pais['PORCENTAJE'] = (ventas_por_pais['VENTAS'] / ventas_por_pais['VENTAS'].sum()) * 100

# # Creamos una tabla para mostrarlo
fig = go.Figure(data=[go.Table(header=dict(values=["<b>País</b>", "<b>Número de Ventas</b>", "<b>ECDF (%)</b>"],
                fill_color='lightblue',align='center',font=dict(size=12, color='black'),
                height=30),cells=dict(values=[ventas_por_pais.COUNTRY, ventas_por_pais.VENTAS, round(ventas_por_pais.PORCENTAJE,3)],
               fill_color='lightgray',align='center',font=dict(size=11),height=25))])

fig.update_layout(title_text='Resumen de Ventas por País', title_x=0.5, width=600)
fig.update_traces(columnwidth=[50, 50, 30])
fig.show()

In [33]:
# Coordenadas de los países
coordenadas_paises = {'COUNTRY': ['Portugal', 'Great Britain', 'Spain', 'Hungary', 'Norway',
    'Denmark', 'Netherlands', 'Italy', 'Czech Republic'],
    'Latitude': [39.3999, 55.3781, 40.4637, 47.1625, 60.4720, 56.2639, 52.1326, 41.8719, 49.8171],
    'Longitude': [-8.2245, -3.4360, -3.7492, 19.5033, 8.4689, 9.5018, 5.2913, 12.5674, 15.4730]}

coordenadas_df = pd.DataFrame(coordenadas_paises)
ventas_con_coordenadas = pd.merge(ventas_por_pais, coordenadas_df, on='COUNTRY')

# Crear el scatter map
fig = px.scatter_map(ventas_con_coordenadas,
    lat='Latitude',lon='Longitude',
    color='VENTAS',
    size='VENTAS',  # Tamaño de los puntos en función del número de ventas
    hover_name='COUNTRY',  # Mostrar el nombre del país al pasar el mouse
    color_continuous_scale=px.colors.cyclical.IceFire,  # Escala de color
    size_max=15,  # Tamaño máximo de los puntos
    zoom=4,  # Nivel de zoom
    center=dict(lat=50, lon=10))  # Centro del mapa

# Mostrar el gráfico
fig.show()

> **DISTRIBUCIÓN DE LAS VENTAS POR AÑO Y MES**


Debemos de tener en cuenta que ventas reales solo se produjeron en los años 2023 y 2024, por eso en los siguientes gráficos no se verá representado el año 2025, pues no contiene ningun valor de venta real.

In [34]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Nº de Ventas Mensuales', 'nº de Ventas Mensuales'))

# Filtramos las ventas totales por mes y tambien del años, para poder comparar
data_ventas_2023 = data[(data['SCENARIO'] == 'actual') & (data['YEAR'] == 2023)]
data_ventas_2024 = data[(data['SCENARIO'] == 'actual') & (data['YEAR'] == 2024)]

ventas_2023 = data_ventas_2023.groupby('MONTH')['AMOUNT'].sum().reset_index(name='VENTAS')
ventas_2024 = data_ventas_2024.groupby('MONTH')['AMOUNT'].sum().reset_index(name='VENTAS')


# Gráfico de Barras
fig.add_trace(go.Bar(x=ventas_2023['MONTH'], y=ventas_2023['VENTAS'], name='2023',
    marker=dict(color='rgba(240, 230, 140, 0.8)', line=dict(color='rgba(0, 0, 0, 0.4)', width=1)),
    width=0.4), row=1, col=1)

fig.add_trace(go.Bar(x=ventas_2024['MONTH'], y=ventas_2024['VENTAS'], name='2024',
    marker=dict(color='rgba(210, 180, 140, 0.8)', line=dict(color='rgba(0, 0, 0, 0.4)', width=1)),width=0.4), row=1, col=1)

# Gráfico de Líneas
fig.add_trace(go.Scatter(x=ventas_2023['MONTH'], y=ventas_2023['VENTAS'],
    mode='lines+markers', name='2023',
    line=dict(color='rgba(240, 230, 140, 0.8)', width=2),
    marker=dict(size=8, symbol='circle')), row=1, col=2)

fig.add_trace(go.Scatter(
    x=ventas_2024['MONTH'], y=ventas_2024['VENTAS'],
    mode='lines+markers', name='2024',
    line=dict(color='rgba(210, 180, 140, 0.8)', width=2),
    marker=dict(size=8, symbol='square')), row=1, col=2)

# Actualizar el layout
fig.update_layout(title='Comparación de Ventas Mensuales por Año',
    xaxis_title='Mes',yaxis_title='Número de Ventas',
    legend_title='Año',hovermode='x unified', height=500)

fig.show()

> **DISTRIBUCIÓN DE LAS VENTAS POR MARCA**

In [35]:
data_actual = data[(data['SCENARIO'] == 'actual')]

ventas_por_marca = data_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index(name='VENTAS')
ventas_por_marca['ECDF'] = ventas_por_marca['VENTAS'].cumsum() / ventas_por_marca['VENTAS'].sum() * 100


fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=("Gráfico de Pareto de Ventas por País", "Distribución de Ventas por País"))

# Gráfico de barras --> Pareto
fig.add_trace(go.Bar(x=ventas_por_marca['SUBBRAND'],y=ventas_por_marca['VENTAS'],
        name="Número de Ventas",text=ventas_por_pais['VENTAS'],
        textposition='outside', marker=dict(color='rgba(240, 230, 140, 0.6)')),row=1, col=1)

# Línea del porcentaje acumulado ---> Pareto
fig.add_trace(go.Scatter(x=ventas_por_marca['SUBBRAND'],y=ventas_por_marca['ECDF'],
        name="Porcentaje Acumulado",yaxis="y2", marker=dict(color='rgba(210, 180, 140, 0.8)')),row=1, col=1)

# Gráfico de tarta
fig.add_trace(go.Pie(labels=ventas_por_marca['SUBBRAND'],values=ventas_por_marca['VENTAS'],
        name="Distribución de Ventas",hole=0.4, textinfo='percent+label'),row=1, col=2)
fig.update_layout(title="Análisis Distribución de Ventas por País",
    colorway=["#D2B48C", "#DEB887", "#BC8F8F", "#A0522D", "#8B4513" ,"#D3C6A6"],)

# Configurar el título y los ejes
fig.update_layout(title="Análisis Distribución de Ventas por País",
    xaxis_title="País", yaxis_title="Nº de Ventas",
    yaxis2=dict(title="Porcentaje Acumulado", overlaying='y', side='right'),)

fig.show()

> ## Tendencia y Estacionalidad en Ventas

> **TENDENCIA Y ESTACIONALIDAD DE LAS VENTAS DEL PAÍS CON MENOS VENTAS**

Tal y como hemos estudiado en los apartados anteriores con las distribucion de las ventas por país, sabemos que el país con menos ventas es **ITALIA**.

In [36]:
data_spain = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'actual')].copy()

data_spain.loc[:, 'DATE'] = pd.to_datetime(data_spain[['YEAR', 'MONTH']].assign(DAY=1))
data_spain = data_spain.sort_values(by='DATE')

fig = px.scatter(data_spain, x="DATE", y="AMOUNT", marginal_x="histogram",
                 marginal_y="rug", title="Ganancias Spain: Dispersión de Ganancias por fecha",
                 labels={"DATE": "Fecha", "AMOUNT": "Ganancias de las Ventas"})
fig.show()

**TENDENCIA** --> Para observar bien la tendencia, lo que haremos es obtener la suma de las ganancias en cada mes, y observarlo en un gráfico de lineas, que nos permita evaluar perfectamente la Tendencia de las ganancias a lo largo del tiempo, para España.

In [37]:
# Agrupamos por fecha y sumamos los valores de Ganancias
ventas_por_fecha = data_spain.groupby('DATE')['AMOUNT'].sum().reset_index()

# Graficamos los resultados, con un gráfico de lineas:
fig = px.line(ventas_por_fecha, x='DATE', y='AMOUNT', title='Ganancias por Fecha en España',
              labels={'AMOUNT': 'Total Amount', 'DATE': 'Fecha'})
fig.update_traces(mode='lines+markers')
fig.show()

Aunque no se puede observar una tendencia muy evidente, es cierto que las ganancias de las ventas en España tienen a crecer conforme pasan los meses. Aún así existen periodos tanto de crecimiento como de decrecimiento.

**ESTACIONALIDAD** --> Observando el gráfico de lineas, podemos ver ciertos comportamientos de estacionalidad. Lo que se observa, es que las ventas en los primeros meses del año (enero-mayo) son inferiores, y a partir de junio empiezan a aumentar.

Vamos a utilizar un boxplot, que nos ayude a contrastar esta estacionalidad observada.

In [38]:
data_spain['MONTH_NAME'] = data_spain['MONTH'].apply(lambda x: f'Month {x}')

# Gráfico de Box Plot por mes
fig = px.box(data_spain, x="DATE", y="AMOUNT", title='Distribución de Ganancias por Mes en España',
             labels={'AMOUNT': 'Total Amount', 'MONTH_NAME': 'Mes'},
             points='all')  # Añade todos los puntos para ver la distribución

fig.show()


Tras usar este gráfico, podemos afirmar las suposiciones realizadas a cerca de la estacionalidad.

Al inicio de año, las ganancias son mas bajas y se mantienen asi o crecen muy lentamente. A partir de los meses de mayo-junio, se aprecia un crecimiento en las ventas en ambos años, posiblemente relacionado con la época del año.




> **TENDENCIA Y ESTACIONALIDAD DE LA MARCA CON MAS VENTAS**


Tal y como hemos estudiado en los apartados anteriores con las distribucion de las ventas por marca, sabemos que la marca con mas ventas es **Pepsi Max (L3)**.

Usaremos los mismos graficos que hemos empleado previamente, para evaluar la tendencia y estacionalidad de las ganancias obtenidas con las ventas de la marca Pepsi Max (L3)

In [39]:
data_pepsi = data[(data['SUBBRAND'] == 'Pepsi Max (L3)') & (data['SCENARIO'] == 'actual')].copy()

data_pepsi.loc[:, 'DATE'] = pd.to_datetime(data_pepsi[['YEAR', 'MONTH']].assign(DAY=1))
data_pepsi = data_pepsi.sort_values(by='DATE')

fig = px.scatter(data_pepsi, x="DATE", y="AMOUNT", marginal_x="histogram",
                 marginal_y="rug", title="Ganancias Pepsi Max (L3): Dispersión de Ganancias por fecha",
                 labels={"DATE": "Fecha", "AMOUNT": "Ganancias de las Ventas"}, color_discrete_sequence=["#90EE90"])
fig.show()

In [40]:
ventas_por_fecha = data_pepsi.groupby('DATE')['AMOUNT'].sum().reset_index()

# Graficamos los resultados, con un gráfico de lineas:
fig = px.line(ventas_por_fecha, x='DATE', y='AMOUNT', title='Ganancias por Fecha en España',
              labels={'AMOUNT': 'Total Amount', 'DATE': 'Fecha'})

fig.update_traces(mode='lines+markers',line=dict(color='#90EE90'))
fig.show()

**TENDENCIA** --> En el caso de la evolución de las ganancias obtenidas por las ventas de la marca Pepsi Max, no se observa una tendencia clara, aunque se podría intuir más una tendencia ascendente, conforme pasan los meses.

**ESTACIONALIDAD** --> En cuanto a la estacionalidad, se puede ver como en Enero y Febrero de ambos años, el valor de las ganancias es mucho mas bajo. A partir de ahí hay un crecimiento brusco en los siguientes meses. La evolución de las ganancias durante los siguientes meses del año se mantiene bastante constante.

> ## Análisis y evaluación de las predicciones hechas en España


Para analizar las predicciones en España, vamos a mostrar las ganancias de las predicciones y ventas reales, con un diagrama de dispersión, mostrando en el eje x las fechas (mes y año) y en el eje y las ganacias de la venta, o las ganancias de la predicción.

En este gráfico aparecerán predicción o venta real como variables categóricas que podremos diferenciar por el color.

In [41]:
# Filtramos los datos de ventas reales y predicciones de España
data_forecast = data[(data['SCENARIO'] == 'AI_forecast') & (data['COUNTRY'] == 'Spain')].copy()
data_real = data[(data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')].copy()

# Covertimos los valores month y year en formato fecha
data_forecast['DATE'] = pd.to_datetime(data_forecast[['YEAR', 'MONTH']].assign(DAY=1))
data_real['DATE'] = pd.to_datetime(data_real[['YEAR', 'MONTH']].assign(DAY=1))

# Añadimos una columna categotica TYPE para distinguir predicciones de datos reales
data_forecast['TYPE'] = 'Predicción'
data_real['TYPE'] = 'Real'

# Concatenamos ambos DataFrames
data_combined = pd.concat([data_forecast, data_real])

# Creamos el gráfico de dispersión
fig = px.scatter(data_combined, x="DATE", y="AMOUNT", color="TYPE",
                 title="Ganancias en España: Datos Reales vs Predicciones",
                 labels={"DATE": "Fecha", "AMOUNT": "Ganancias", "TYPE": "Tipo de Datos"},
                 color_discrete_map={"Real": "black", "Predicción": "#FFD700"})

# Mostramos el gráfico
fig.show()

In [42]:
# Calculamos el total de ganancias de predicciones y valores reales por fecha
forecast_by_date = data_forecast.groupby('DATE')['AMOUNT'].sum().reset_index(name='PREDICTION_AMOUNT')
real_by_date = data_real.groupby('DATE')['AMOUNT'].sum().reset_index(name='REAL_AMOUNT')

# Unimos las tablas para poder calcular el error
comparison = pd.merge(forecast_by_date, real_by_date, on='DATE', how='inner')

# Calculamos el error absoluto entre el valor real y la predicción
comparison['ERROR_ABSOLUTO'] = abs(comparison['REAL_AMOUNT'] - comparison['PREDICTION_AMOUNT'])

# Creamos el subplot con dos gráficos: Línea de predicción y real, y Error absoluto
fig = make_subplots(rows=1, cols=2,
    subplot_titles=("Predicciones vs Reales en el Tiempo", "Error Absoluto en el Tiempo"),
    specs=[[{"type": "scatter"}, {"type": "bar"}]])

# Gráfico de línea para valores reales y predicciones
fig.add_trace(go.Scatter(x=comparison['DATE'], y=comparison['REAL_AMOUNT'],
               mode='lines+markers', name='Valores Reales',
               line=dict(color='black', width=2)),row=1, col=1)

fig.add_trace(go.Scatter(x=comparison['DATE'], y=comparison['PREDICTION_AMOUNT'],
               mode='lines+markers', name='Predicciones AI',
               line=dict(color='#FFD700')),row=1, col=1)

# Gráfico de barras para el error absoluto
fig.add_trace(go.Bar(x=comparison['DATE'], y=comparison['ERROR_ABSOLUTO'], name='Error Absoluto',
           marker=dict(color='#FFD700')),row=1, col=2)

# Configuración general de layout
fig.update_layout(title="Análisis de Predicciones AI en el Tiempo",
    xaxis_title="Fecha",yaxis_title="Ganancias (Amount)",
    xaxis2_title="Fecha",yaxis2_title="Error Absoluto")

fig.show()